In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import numpy as np

In [2]:
listings=pd.read_csv("../source/clean/listings_all_clean.csv")
listings

,Feature,Type,Bath,Bed,Car,Suburb,Postcode,Address,Latitude,Longitude,Floorplan,SoldMethod,SoldDate,SoldPrice,Year,Offence Count,Local Government Area,"Rate per 100,000 population"
0,['Heating'],ApartmentUnitFlat,1,2,0,MELBOURNE,3000,"514/9 Degraves Street, Melbourne",-37.817764,144.965652,True,SoldByPrivateTreaty,1/1/20,585000,2020,19557,Melbourne,19054.381440
1,"['Bath', 'Heating']",ApartmentUnitFlat,1,1,0,MELBOURNE,3000,"114/318 Little Bourke Street, Melbourne",-37.812930,144.963100,True,SoldByPrivateTreaty,3/1/20,380000,2020,19557,Melbourne,19054.381440
2,"['BuiltInWardrobes', 'Intercom', 'Heating', 'D...",ApartmentUnitFlat,1,1,0,MELBOURNE,3000,"605/115 Swanston Street, Melbourne",-37.814995,144.966034,False,SoldByPrivateTreaty,6/1/20,295000,2020,19557,Melbourne,19054.381440
3,"['SwimmingPool', 'Intercom', 'Heating']",ApartmentUnitFlat,2,2,0,MELBOURNE,3000,"1506/27 Lt Collins Street, Melbourne",-37.812725,144.972626,True,SoldByPrivateTreaty,7/1/20,596000,2020,19557,Melbourne,19054.381440
4,"['Ensuite', 'SwimmingPool']",ApartmentUnitFlat,2,2,0,MELBOURNE,3000,"53/24-38 Little Bourke Street, Melbourne",-37.810560,144.971512,True,SoldByPrivateTreaty,7/1/20,587000,2020,19557,Melbourne,19054.381440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302592,"['Ensuite', 'Heating', 'Shed']",House,2,4,3,KOO WEE RUP,3981,"37 Beatties Road, Koo Wee Rup",-38.215157,145.472046,False,SoldByPrivateTreaty,8/8/11,550000,2011,181,Baw Baw,4489.617184
302593,"['SecureParking', 'Dishwasher']",House,3,4,2,BAYLES,3981,"175 School Road, Bayles",-38.194090,145.576263,False,NotStated,12/12/11,868000,2011,181,Baw Baw,4489.617184
302594,"['SecureParking', 'Shed']",House,2,4,2,BAYLES,3981,"175 School Road, Bayles",-38.194090,145.576263,False,SoldByPrivateTreaty,12/12/11,868000,2011,181,Baw Baw,4489.617184
302595,['Heating'],House,1,3,0,PIONEER BAY,3984,"9 Whiting Street, Pioneer Bay",-38.380130,145.546814,False,SoldByPrivateTreaty,16/7/11,190000,2011,234,Bass Coast,6721.132537


In [3]:
listings=listings.drop(["Feature","Postcode","Floorplan"],axis=1)
listings=pd.get_dummies(listings, columns=["SoldMethod"])
listings.columns

Index(['Type', 'Bath', 'Bed', 'Car', 'Suburb', 'Address', 'Latitude',
       'Longitude', 'SoldDate', 'SoldPrice', 'Year', 'Offence Count',
       'Local Government Area', 'Rate per 100,000 population',
       'SoldMethod_NotStated', 'SoldMethod_SoldByAuction',
       'SoldMethod_SoldByPrivateTreaty', 'SoldMethod_SoldPriorToAuction'],
      dtype='object')

In [4]:
listings=pd.DataFrame(listings.groupby(["Suburb","Local Government Area","Year","Type"]).agg(
    {'Address':'count',
     'Bath':'mean',
     'Bed':'mean',
     'Car':'mean',
     'Latitude':'mean',
     'Longitude':'mean',
     'SoldPrice':'mean',
     'Offence Count':'mean',
     'Rate per 100,000 population':'mean',
     'SoldMethod_SoldByAuction':'sum',
     'SoldMethod_SoldByPrivateTreaty':'sum',
     'SoldMethod_SoldPriorToAuction':'sum'
    }))
listings=listings.reset_index(drop=False)
listings["Bath"]=listings["Bath"].astype(int)
listings["Bed"]=listings["Bed"].astype(int)
listings["Car"]=listings["Car"].astype(int)
listings["SoldPrice"]=listings["SoldPrice"].astype(int)
listings["offence_count"]=listings["Offence Count"].astype(int)
listings["crime_rate_lga"]=listings["Rate per 100,000 population"].astype(int)
listings=listings.drop(["Offence Count","Rate per 100,000 population"],axis=1)
listings=listings.rename(columns={'Address':'count'})
listings=listings[listings["count"]>1]
listings

,Suburb,Local Government Area,Year,Type,count,Bath,Bed,Car,Latitude,Longitude,SoldPrice,SoldMethod_SoldByAuction,SoldMethod_SoldByPrivateTreaty,SoldMethod_SoldPriorToAuction,offence_count,crime_rate_lga
0,ABBOTSFORD,Yarra,2011,ApartmentUnitFlat,6,1,1,1,-37.804040,144.998246,694166,3,3,0,1356,13748
1,ABBOTSFORD,Yarra,2011,House,36,1,2,0,-37.804425,144.996713,819388,20,16,0,1356,13748
3,ABBOTSFORD,Yarra,2012,ApartmentUnitFlat,11,1,1,1,-37.804185,144.999155,497772,6,5,0,908,13467
4,ABBOTSFORD,Yarra,2012,House,46,1,2,0,-37.803978,144.996714,817402,33,13,0,908,13467
6,ABBOTSFORD,Yarra,2013,ApartmentUnitFlat,14,1,1,1,-37.803025,145.000240,550214,6,8,0,748,12654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10002,YELLINGBO,Yarra Ranges,2020,House,2,2,4,6,-37.820441,145.510498,1050000,0,2,0,614,5452
10004,YERING,Yarra Ranges,2015,House,8,2,4,4,-37.695804,145.355445,1388000,0,8,0,129,4609
10005,YERING,Yarra Ranges,2017,House,2,2,4,6,-37.695946,145.351151,1600000,0,2,0,156,5444
10006,YERING,Yarra Ranges,2018,House,2,3,5,2,-37.696495,145.356216,2008000,0,2,0,129,4978


In [5]:
listings.Type.unique()

array(['ApartmentUnitFlat', 'House', 'Townhouse'], dtype=object)

In [6]:
building_type=pd.DataFrame(
    {"type":listings.Type.unique()})
building_type

,type
0,ApartmentUnitFlat
1,House
2,Townhouse


In [7]:
building_type.to_csv("../database/building.csv",index=False)

In [8]:
listings["Local Government Area"].unique()

array(['Yarra', 'Moonee Valley', 'Brimbank', 'Port Phillip', 'Darebin',
       'Hobsons Bay', 'Stonnington', 'Nillumbik', 'Maribyrnong',
       'Macedon Ranges', 'Boroondara', 'Kingston', 'Hume', 'Cardinia',
       'Greater Geelong', 'Murrindindi', 'Glen Eira', 'Baw Baw', 'Knox',
       'Bayside', 'Yarra Ranges', 'Banyule', 'Casey', 'Mitchell',
       'Whitehorse', 'Melton', 'Moreland', 'Manningham', 'Monash',
       'Melbourne', 'Bass Coast', 'Maroondah', 'Moorabool',
       'Greater Dandenong', 'Wyndham', 'Whittlesea'], dtype=object)

In [9]:
lga=pd.DataFrame(
    {"lga":listings["Local Government Area"].unique()})
lga.head()

,lga
0,Yarra
1,Moonee Valley
2,Brimbank
3,Port Phillip
4,Darebin


In [10]:
lga.to_csv("../database/lga.csv",index=False)

In [11]:
listings["Suburb"]=[x.lower() for x in listings["Suburb"]]

In [12]:
suburb=listings[["Suburb","Local Government Area"]]
suburb=suburb.drop_duplicates(subset=['Suburb'],keep='first')
suburb.columns=['suburb','lga']
suburb.head()

,suburb,lga
0,abbotsford,Yarra
30,aberfeldie,Moonee Valley
59,airport west,Brimbank
89,albanvale,Brimbank
103,albert park,Port Phillip


In [13]:
suburb.to_csv("../database/suburb.csv",index=False)

In [14]:
year=pd.DataFrame({"year":listings["Year"].unique()})
year.head()

,year
0,2011
1,2012
2,2013
3,2014
4,2015


In [15]:
year.to_csv("../database/year.csv",index=False)

In [16]:
offence_rate=pd.DataFrame(listings.groupby(["Year","Local Government Area"])["crime_rate_lga"].mean())
offence_rate=offence_rate.reset_index()
offence_rate.columns=['year','lga','crime_rate']
offence_rate.head()

,year,lga,crime_rate
0,2011,Banyule,5343
1,2011,Baw Baw,4489
2,2011,Bayside,3428
3,2011,Boroondara,3469
4,2011,Brimbank,8650


In [17]:
offence_rate.to_csv("../database/offence_rate.csv",index=False)

In [18]:
listing=listings[["count","Type","Bath","Bed","Car","Suburb","Latitude","Longitude","SoldMethod_SoldByAuction",\
                  "SoldMethod_SoldByPrivateTreaty","SoldMethod_SoldPriorToAuction","Year","SoldPrice","offence_count"]]
listing.columns=["count","type","bath","bed","car","suburb","latitude","longitude","auction","private_treaty",\
                 "prior_to_auction","year","price","offence_count"]

In [19]:
listing.head()

,count,type,bath,bed,car,suburb,latitude,longitude,auction,private_treaty,prior_to_auction,year,price,offence_count
0,6,ApartmentUnitFlat,1,1,1,abbotsford,-37.804040,144.998246,3,3,0,2011,694166,1356
1,36,House,1,2,0,abbotsford,-37.804425,144.996713,20,16,0,2011,819388,1356
3,11,ApartmentUnitFlat,1,1,1,abbotsford,-37.804185,144.999155,6,5,0,2012,497772,908
4,46,House,1,2,0,abbotsford,-37.803978,144.996714,33,13,0,2012,817402,908
6,14,ApartmentUnitFlat,1,1,1,abbotsford,-37.803025,145.000240,6,8,0,2013,550214,748


In [20]:
listing.to_csv("../database/listing.csv",index=False)